<a href="https://colab.research.google.com/github/JanKotynia/SalesDataAnalysis/blob/main/Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [223]:
import kagglehub
import pandas as pd
import numpy as np
import os

path = kagglehub.dataset_download("ikramshah512/amazon-products-sales-dataset-42k-items-2025")

print(os.listdir(path))
file_path = os.path.join(path, "amazon_products_sales_data_uncleaned.csv")
file_path2 = os.path.join(path, "amazon_products_sales_data_cleaned.csv")

Using Colab cache for faster access to the 'amazon-products-sales-dataset-42k-items-2025' dataset.
['amazon_products_sales_data_cleaned.csv', 'amazon_products_sales_data_uncleaned.csv']


In [224]:
df = pd.read_csv(file_path, encoding="utf-8")
df2 = pd.read_csv(file_path2, encoding="utf-8")

In [225]:
df.head(5)

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


## Preparing the dataset

In [226]:
df.isnull().sum()

,0
title,0
rating,1024
number_of_reviews,1024
bought_in_last_month,3217
current/discounted_price,11749
price_on_variant,0
listed_price,0
is_best_seller,0
is_sponsored,0
is_couponed,0


In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     42675 non-null  object
 1   rating                    41651 non-null  object
 2   number_of_reviews         41651 non-null  object
 3   bought_in_last_month      39458 non-null  object
 4   current/discounted_price  30926 non-null  object
 5   price_on_variant          42675 non-null  object
 6   listed_price              42675 non-null  object
 7   is_best_seller            42675 non-null  object
 8   is_sponsored              42675 non-null  object
 9   is_couponed               42675 non-null  object
 10  buy_box_availability      28022 non-null  object
 11  delivery_details          30955 non-null  object
 12  sustainability_badges     3408 non-null   object
 13  image_url                 42675 non-null  object
 14  product_url           

In [228]:
df.shape

(42675, 16)

removing unnecessary columns

In [229]:
df.drop(['price_on_variant','buy_box_availability',	'sustainability_badges', 'image_url',	'product_url', 'collected_at'], axis=1,inplace=True)

identification and removing rows without prices

In [230]:
df.drop(df[(df['listed_price'] == 'No Discount') & (df['current/discounted_price'].isna())].index, inplace=True)

setting 'No Discount' rows to 'current/discounted_price'

In [231]:
df.loc[df['listed_price'] == 'No Discount', 'listed_price'] = df.loc[df['listed_price'] == 'No Discount', 'current/discounted_price']

processing numeric columns, assigning them to other columns, and changing data types

In [232]:
df['listed_price'] = df['listed_price'].str.replace(r'^[^\d]+', '', regex=True)

df.loc[df['current/discounted_price'].isna(), 'current/discounted_price'] = df.loc[df['current/discounted_price'].isna(),'listed_price']

In [233]:
df['current/discounted_price'] = df['current/discounted_price'].str.replace(r',', '', regex=True)
df['number_of_reviews'] = df['number_of_reviews'].str.replace(r',', '', regex=True)
df['listed_price'] = df['listed_price'].str.replace(r',', '', regex=True)

df['listed_price'] = df['listed_price'].astype(float)
df['current/discounted_price'] = df['current/discounted_price'].astype(float)
df['number_of_reviews'] = df['number_of_reviews'].astype(float)

filling null values

In [234]:
df.loc[df['number_of_reviews'].isna(), 'number_of_reviews'] = 0
df.loc[df['bought_in_last_month'].isna(), 'bought_in_last_month'] = 0
df.loc[df['bought_in_last_month'] == '', 'bought_in_last_month'] = 0
df.loc[df['rating'].isna(), 'rating'] = 0
df.loc[df['delivery_details'].isna(), 'delivery_details'] = 'No data'

In [235]:
df['bought_in_last_month'] = df['bought_in_last_month'].str.replace(r'[^0-9K]+$', '', regex=True)

a lot of strange values

In [236]:
df['bought_in_last_month'].value_counts().tail(10)

,count
bought_in_last_month,
ESRB Rating: Everyone 10,1
($54.26$54.26,1
($828.56$828.56,1
($46.81$46.81,1
"($1,080.00$1,080.00",1
($433.33$433.33,1
"($1,315.78$1,315.78",1
"($4,813.00$4,813.00/100",1
($5.27$5.27,1


drop rows with strange values

In [237]:
df = df[df['bought_in_last_month'].astype(str).str.match(r'^[0-9K]+$')]

final adjustments and column type modification

In [238]:
mask = df['bought_in_last_month'].str[-1] == 'K'
df.loc[mask, 'bought_in_last_month'] = df.loc[mask, 'bought_in_last_month'].apply(lambda x: np.double(str(x)[:-1]) * 1000)

In [239]:
df['bought_in_last_month'] = df['bought_in_last_month'].astype(float)

In [240]:
df.isna().sum()

,0
title,0
rating,0
number_of_reviews,0
bought_in_last_month,0
current/discounted_price,0
listed_price,0
is_best_seller,0
is_sponsored,0
is_couponed,0
delivery_details,0


changing rating datatype

In [241]:
df['rating'] = df['rating'].apply(lambda x : str(x).split(' ')[0])
df['rating'] = df['rating'].astype(float)

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25237 entries, 0 to 42674
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     25237 non-null  object 
 1   rating                    25237 non-null  float64
 2   number_of_reviews         25237 non-null  float64
 3   bought_in_last_month      25237 non-null  float64
 4   current/discounted_price  25237 non-null  float64
 5   listed_price              25237 non-null  float64
 6   is_best_seller            25237 non-null  object 
 7   is_sponsored              25237 non-null  object 
 8   is_couponed               25237 non-null  object 
 9   delivery_details          25237 non-null  object 
dtypes: float64(5), object(5)
memory usage: 2.1+ MB


In [243]:
df['delivery_details'].value_counts().head(50)

,count
delivery_details,
"Delivery Mon, Sep 1",5023
"Delivery Thu, Sep 4",3920
"Delivery Thu, Sep 11",2894
"Delivery Tue, Sep 2",2607
"Delivery Wed, Sep 3",1649
"Delivery Tue, Sep 9",1011
"FREE delivery Thu, Sep 4Or fastest delivery Tomorrow, Aug 31",930
"FREE delivery Wed, Sep 3Or fastest delivery Sun, Aug 31",896
"FREE delivery Wed, Sep 3 on $35 of items shipped by AmazonOr fastest delivery Sun, Aug 31",768


Cutting out the paid delivery date and setting NaN for fields without a specified delivery date

In [244]:
df['delivery_details'] = df['delivery_details'].astype(str).str.extract(r'([A-Z][a-z]{2}(?:[-\s]*\d+)*)$')
df.loc[df['delivery_details'].isna(), 'delivery_details'] = "No data"


In [245]:
df['delivery_details'].head(50)

,delivery_details
0,Sep 1
1,Aug 29
2,Sep 1
10,Aug 29
12,Sep 1
13,Sep 1
14,Aug 29
16,Sep 1 - 11
17,Aug 29
18,Aug 29


new column with date format data

In [246]:
df['delivery_date'] = pd.to_datetime(
    df['delivery_details'] + ' 2025',
    format='%b %d %Y',
    errors='coerce'
)

In [247]:
df.loc[df['delivery_date'].isna(),'delivery_date'] = pd.to_datetime(
    df.loc[mask, 'delivery_details']
      .str.extract(r'([A-Z][a-z]{2}\s*\d{1,2})')[0]
      .add(' 2025'),
    format='%b %d %Y',
    errors='coerce'
)

In [248]:
mask = df['delivery_details'] == "No data"
df.loc[mask,'delivery_date'] = np.nan

In [249]:
df.drop(columns='delivery_details', inplace=True)

changing sponsored column type

In [250]:
df['is_sponsored'].value_counts()

,count
is_sponsored,
Organic,21636
Sponsored,3601


In [251]:
df['is_sponsored'] = np.where(df['is_sponsored'] == 'Organic', False, True)

changing is_couponed column type

In [252]:
df['is_couponed'].value_counts()

,count
is_couponed,
No Coupon,24116
Save 15% with coupon,398
Save 10% with coupon,193
Save 5% with coupon,190
Save $100.00 with coupon,96
Save $25.00 with coupon,65
Save $30.00 with coupon,48
Save $69.00 with coupon,31
Save 8% with coupon,18


Set "No Coupon" to 0 and extract only the numeric value from the coupon.

In [253]:
df.loc[df['is_couponed'] == 'No Coupon','is_couponed'] = float(0.0)
df['is_couponed'] = np.where(df['is_couponed'] == 'No Coupon', float(0.0), df['is_couponed'].str.split().str[1])

**Final column types:**

In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25237 entries, 0 to 42674
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     25237 non-null  object        
 1   rating                    25237 non-null  float64       
 2   number_of_reviews         25237 non-null  float64       
 3   bought_in_last_month      25237 non-null  float64       
 4   current/discounted_price  25237 non-null  float64       
 5   listed_price              25237 non-null  float64       
 6   is_best_seller            25237 non-null  object        
 7   is_sponsored              25237 non-null  bool          
 8   is_couponed               1121 non-null   object        
 9   delivery_date             23822 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(5), object(3)
memory usage: 1.9+ MB


### Dataset Analysis